In [ ]:
import evaluate
from evaluate import load
import pandas as pd

In [ ]:
bleu = evaluate.load("bleu")
meteor = evaluate.load('meteor')
rouge = evaluate.load('rouge')
bertscore = load("bertscore")
frugalscore = evaluate.load("frugalscore")

In [ ]:
import pandas as pd

# Load the cleaned data file
input_file = 'path_to_directory'
df = pd.read_csv(input_file)

# Initialize lists to store results and processed indices
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
rougeLsum_scores = []
bertscore_precision = []
bertscore_recall = []
bertscore_f1 = []
meteor_scores = []
processed_indices = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Validate and clean data
    if pd.isna(row['generated_report']) or pd.isna(row['original_report']):
        continue  # Skip rows with missing data
    if not isinstance(row['generated_report'], str) or not isinstance(row['original_report'], str):
        continue  # Skip rows with non-string data
    if row['generated_report'].strip() == "" or row['original_report'].strip() == "":
        continue  # Skip rows with empty strings
    
    predictions = [row['generated_report']]
    references = [row['original_report']]
    
    try:
        # Compute METEOR
        meteor_result = meteor.compute(predictions=predictions, references=references)
        meteor_scores.append(meteor_result['meteor'])
        
        # Compute BERTScore
        bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")
        bertscore_precision.append(bertscore_result['precision'][0])
        bertscore_recall.append(bertscore_result['recall'][0])
        bertscore_f1.append(bertscore_result['f1'][0])
        
        # Compute ROUGE
        rouge_result = rouge.compute(predictions=predictions, references=references)
        rouge1_scores.append(rouge_result['rouge1'])
        rouge2_scores.append(rouge_result['rouge2'])
        rougeL_scores.append(rouge_result['rougeL'])
        rougeLsum_scores.append(rouge_result['rougeLsum'])
        
        # Keep track of processed rows
        processed_indices.append(index)
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue

# Create a new DataFrame with results using only the processed indices
results_df = pd.DataFrame({
    'image_name': df.loc[processed_indices, 'image_name'].values,
    'rouge1': rouge1_scores,
    'rouge2': rouge2_scores,
    'rougeL': rougeL_scores,
    'rougeLsum': rougeLsum_scores,
    'bertscore_precision': bertscore_precision,
    'bertscore_recall': bertscore_recall,
    'bertscore_f1': bertscore_f1,
    'meteor': meteor_scores
})

# Save the results to a new CSV file
output_file = 'path_to_directory'
results_df.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")